## 3.0 LLMs and Chat Models

In [2]:
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()

from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

# # OpenAI API 키 확인 (디버깅용)
# print(os.getenv("OPENAI_API_KEY"))

llm = OpenAI(model="gpt-3.5-turbo-instruct")
chat = ChatOpenAI()

a = llm.predict('How many planets are there?')
b = chat.predict('How many planets are there?')

a,b

('\n\nThere are currently eight recognized planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.',
 'There are eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.')

## 3.1 Predict Messages

In [3]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0.1)

In [4]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(
        content = 'You are a geography expert. And you only reply on {language}'), 
    AIMessage(
        content = 'Ciao, mi chiamo {name}!'), 
    HumanMessage(
        content = 'What is the distance between {country_a} and {country_b}. Also, what is your name?'
    )]

chat.predict_messages(messages)

AIMessage(content="Mi dispiace, non posso calcolare la distanza esatta tra due paesi senza informazioni specifiche sulla posizione esatta all'interno di ciascun paese. Il mio nome è Assistente Virtuale. Posso aiutarti con altre domande sulla geografia?")

## 3.2 Prompt Templates

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template(
    'What is the distance between {country_a} and {country_b}'
    )

prompt = template.format(country_a = 'Mexico', country_b = 'Thailand')

chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 9,500 miles (15,300 kilometers) when measured in a straight line.'

In [7]:
template = ChatPromptTemplate.from_messages([
    ('system','You are a geography expert. And you only reply on {language}'),
    ('ai', 'Ciao, mi chiamo {name}!'),
    ('human', 'What is the distance between {country_a} and {country_b}. Also, what is your name?')
    ])

prompt = template.format_messages(
    language = 'Greek',
    name = 'Socrates',
    country_a = 'Mexico',
    country_b = 'Thailand'
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα.')

## 3.3 OutputParser and LCEL

In [12]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    
    def parse(self, text):
        items = text.strip().split(',')
        return list(map(str.strip, items))
    
p = CommaOutputParser()

p.parse('Hello,How,are,you?')


['Hello', 'How', 'are', 'you?']

In [17]:
template = ChatPromptTemplate.from_messages([
    ('system', 
     "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items}. Do not reply with anything else."),
    ('human', '{question}'),
    ]
)

prompt = template.format_messages(
    max_items = 10, 
    question = 'what are the planets?'
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)

['Mercury', 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune']

In [18]:
chain = template | chat | CommaOutputParser()

chain.invoke({
    'max_items' : 5,
    'question' : 'what are the pokemons?'
})

['Pikachu', 'Charizard', 'Bulbasaur', 'Squirtle', 'Jigglypuff']

## 3.4 Chaining Chains

In [23]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks= [StreamingStdOutCallbackHandler()]
    )

chef_prompt = ChatPromptTemplate.from_messages([
    ('system','You are a world-class international chef. You create easty to follow recipes for any types of cuisine with easy to find ingredients.'),
    ('human', 'I want to cook {cuisine} food.'),
])

chef_chain = chef_prompt | chat

In [24]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

final_chain = {'recipe' : chef_chain} | veg_chain

final_chain.invoke({
    'cuisine' : 'indian',
})

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic and popular dish - Chicken Tikka Masala. Here's a simple recipe for you to try at home:

Chicken Tikka Masala

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tbsp lemon juice
- 2 tsp ground cumin
- 2 tsp ground coriander
- 1 tsp turmeric
- 1 tsp paprika
- 1 tsp garam masala
- 1 tsp chili powder (adjust to taste)
- Salt to taste
- 2 tbsp vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1-inch piece of ginger, grated
- 1 can (14 oz) crushed tomatoes
- 1 cup heavy cream
- Fresh cilantro, chopped (for garnish)

Instructions:
1. In a bowl, mix together yogurt, lemon juice, cumin, coriander, turmeric, paprika, garam masala, chili powder, and salt. Add the chicken pieces and coat them well with the marinade. Cover and refrigerate for at least 1 hour, or overnight for best results.

2. Preheat the oven to 400°

AIMessageChunk(content="For a vegetarian version of Chicken Tikka Masala, you can replace the chicken with a plant-based alternative such as tofu or paneer. Here's how you can prepare these alternatives:\n\n1. **Tofu**: \n   - Use firm or extra firm tofu for this recipe.\n   - Drain the tofu and press it to remove excess water. You can do this by wrapping the tofu block in a clean kitchen towel or paper towels and placing a heavy object on top for about 15-30 minutes.\n   - Cut the tofu into bite-sized cubes and follow the same marinating and baking instructions as you would for the chicken. Tofu absorbs flavors well, so it will taste delicious with the spice marinade.\n\n2. **Paneer**:\n   - Paneer is a type of Indian cheese that holds its shape well when cooked.\n   - Cut the paneer into cubes and lightly pan-fry or grill them before adding to the sauce. This will help the paneer hold its shape and develop a nice texture.\n   - You can also marinate the paneer cubes in the yogurt and